In [2]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [19]:
meta = pd.read_excel("__Articles With Raw Tags V2.xlsx", encoding="utf8", 
                     error_bad_lines=False)
meta = meta.replace(np.nan, '', regex=True)

In [20]:
meta = meta[~(meta['Tags'].str.len() == 0)]

In [21]:
meta.head()

,ID,Titolo,Trend,Tags
0,c812e0a3418d624ae60df3ec9e19d184,pandemic drone spot infected,Robotics and Human/Machine Hybridization,Tech: Drones|||||||||||||||||||||
1,0e2ef6bd7619374e0531d1551ab2141e,census data anonymous expected,Privacy and Security by design,||Vertical: Advertising|Vertical: Advertising|...
2,8602d8791a1754506303dcfd9a7b79e3,common pandemic scams highlighted domain name ...,Cyber Defense and Fight against Fakes,|||||||||||||||||||||
3,2157ac6d12e063b6a38c5c0491e0b322,questions rapidly unfolding future smart fabrics,Workforce Transformation,|||||||||||Tag: Future||||||||||
4,89390733d3f9e1f30bae53de5e2d8a04,types robots playing vital roles covid,New Normal after COVID-19,|||||||||||||||||||||


In [22]:
meta['Tags'][1]

'||Vertical: Advertising|Vertical: Advertising||Company: Adobe||Vertical: Advertising||||||||||||||'

In [23]:
def string_to_commaseparated(text):
    str_list = text.split("|")
    str_list = list(filter(None, str_list))
    str_list = list(set(str_list))
    
    str_list = ", " . join(str_list)
    
    return str_list
    
string_to_commaseparated(meta['Tags'][1])

'Company: Adobe, Vertical: Advertising'

In [24]:
meta['Tags'] = meta['Tags'].apply(lambda x: string_to_commaseparated(x))

In [25]:
meta.head()

,ID,Titolo,Trend,Tags
0,c812e0a3418d624ae60df3ec9e19d184,pandemic drone spot infected,Robotics and Human/Machine Hybridization,Tech: Drones
1,0e2ef6bd7619374e0531d1551ab2141e,census data anonymous expected,Privacy and Security by design,"Company: Adobe, Vertical: Advertising"
2,8602d8791a1754506303dcfd9a7b79e3,common pandemic scams highlighted domain name ...,Cyber Defense and Fight against Fakes,
3,2157ac6d12e063b6a38c5c0491e0b322,questions rapidly unfolding future smart fabrics,Workforce Transformation,Tag: Future
4,89390733d3f9e1f30bae53de5e2d8a04,types robots playing vital roles covid,New Normal after COVID-19,


In [26]:
def get_tags_types(tags):
    tag_types = []
    for tag in tags:
        t1 = tag.split(",")[0]
        if len(t1):
            t2 = t1.split(":")[0]
            tag_types.append(t2)
            
    tag_types = list(map(str.strip, tag_types))
    tag_types = list(filter(None, tag_types))
    tag_types = list(set(tag_types))
    
    return tag_types
    
get_tags_types(meta['Tags'])

['Tech', 'Company', 'Vertical', 'Application', 'Location', 'Tag']

In [27]:
def split_tags_in_columns(tags, column):
    tag_list = tags.split(",")
    for t in tag_list:
        if t.split(":")[0].strip() == column:
            return t.split(":")[1].strip()
        
    return ''
    
split_tags_in_columns(meta['Tags'][1], 'Vertical')

'Advertising'

In [28]:
for tt in get_tags_types(meta['Tags']):
    meta[tt] = meta['Tags'].apply(lambda x: split_tags_in_columns(x, tt))

In [29]:
meta.head()

,ID,Titolo,Trend,Tags,Tech,Company,Vertical,Application,Location,Tag
0,c812e0a3418d624ae60df3ec9e19d184,pandemic drone spot infected,Robotics and Human/Machine Hybridization,Tech: Drones,Drones,,,,,
1,0e2ef6bd7619374e0531d1551ab2141e,census data anonymous expected,Privacy and Security by design,"Company: Adobe, Vertical: Advertising",,Adobe,Advertising,,,
2,8602d8791a1754506303dcfd9a7b79e3,common pandemic scams highlighted domain name ...,Cyber Defense and Fight against Fakes,,,,,,,
3,2157ac6d12e063b6a38c5c0491e0b322,questions rapidly unfolding future smart fabrics,Workforce Transformation,Tag: Future,,,,,,Future
4,89390733d3f9e1f30bae53de5e2d8a04,types robots playing vital roles covid,New Normal after COVID-19,,,,,,,


In [31]:
meta.to_csv('2 - Articles With Tags V2.csv', sep=";")